In [ ]:
#install.packages("h2o")
library(h2o)

Loading the data set

In [15]:
car_dataset_loc <- "Desktop/training.csv"
car_dataset <- read.csv(car_dataset_loc)

Preprocessing the data

In [16]:
car_dataset$RefId <- NULL
car_dataset$PurchDate <- NULL
car_dataset$WheelTypeID <- NULL


car_dataset <- data.frame(lapply(car_dataset, as.character), stringsAsFactors = FALSE)
car_dataset[car_dataset == ""] <- NA
car_dataset[car_dataset == "NULL"] <- NA


categorical_var <- c("IsBadBuy", "Auction", "Make", "Model", "Trim", "SubModel", "Color", "Transmission",
               "WheelType", "Nationality", "Size", "TopThreeAmericanName", "PRIMEUNIT", "AUCGUART", "BYRNO",
               "VNZIP1", "VNST", "IsOnlineSale")
car_dataset[categorical_var] <- lapply(car_dataset[categorical_var], as.factor)


numerical_var <- c("VehYear", "VehicleAge", "VehOdo", "MMRAcquisitionAuctionAveragePrice", "MMRAcquisitionAuctionCleanPrice",
                "MMRAcquisitionRetailAveragePrice", "MMRAcquisitonRetailCleanPrice", "MMRCurrentAuctionAveragePrice", 
                "MMRCurrentAuctionCleanPrice", "MMRCurrentRetailAveragePrice", "MMRCurrentRetailCleanPrice",
                "VehBCost", "WarrantyCost")
car_dataset[numerical_var] <- lapply(car_dataset[numerical_var], as.numeric)



Applying random forest prediction

In [17]:
h2o.no_progress()
h2o.init(nthreads = -1)
car_dataset_h2o <- as.h2o(car_dataset)
car_dataset_h2o <- h2o.splitFrame(car_dataset_h2o, ratios = 0.80, destination_frames = c("train", "test"))
names(car_dataset_h2o) <- c("train", "test")
#write.csv(as.data.frame(car_dataset_h2o$test), "testset.csv")
inputModel <- names(car_dataset[-1])

rf_car_dataset <- h2o.randomForest(training_frame = car_dataset_h2o$train, validation_frame = car_dataset_h2o$test, x = inputModel, y = "IsBadBuy", ntrees = 100, stopping_rounds = 2)

rf_car_dataset@model$validation_metric


 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         4 hours 12 minutes 
    H2O cluster version:        3.10.4.6 
    H2O cluster version age:    10 days  
    H2O cluster name:           H2O_started_from_R_vijaykulkarni_mus090 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   0.68 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:                  R version 3.3.2 (2016-10-31) 



H2OBinomialMetrics: drf
** Reported on validation data. **

MSE:  0.08905486
RMSE:  0.2984206
LogLoss:  0.312403
Mean Per-Class Error:  0.3447942
AUC:  0.7477289
Gini:  0.4954578

Confusion Matrix (vertical: actual; across: predicted) for F1-optimal threshold:
           0    1    Error         Rate
0      11800 1044 0.081283  =1044/12844
1       1084  698 0.608305   =1084/1782
Totals 12884 1742 0.145494  =2128/14626

Maximum Metrics: Maximum metrics at their respective thresholds
                        metric threshold    value idx
1                       max f1  0.227548 0.396141 178
2                       max f2  0.103883 0.479468 280
3                 max f0point5  0.551259 0.528476  66
4                 max accuracy  0.570902 0.898674  58
5                max precision  0.772583 1.000000   0
6                   max recall  0.004078 1.000000 397
7              max specificity  0.772583 1.000000   0
8             max absolute_mcc  0.551259 0.389085  66
9   max min_per_class_accura

Write the variable importance attached to each column name to a csv file

In [10]:
varimp <- h2o.varimp(rf_car_dataset)
write.csv(as.data.frame(varimp),"Desktop/varimp.csv")

Obtain the predicted values

In [18]:
test=as.data.frame(car_dataset_h2o$test)
predictedVal=as.data.frame(predict(rf_car_dataset, car_dataset_h2o$test))
test$predictedVal=predictedVal$predict

Assign probabilities to predicted values

In [20]:
test$p0 <- predictedVal$p0
test$p1 <- predictedVal$p1
write.csv(test,"testing-file.csv")